In [1]:
import pandas as pd
import csv
from anytree import Node, Walker
import re
import ast
import swifter

In [2]:
eu_tags = pd.read_csv("filtered_eu.csv")
eu_tags["name_list"] = eu_tags["name_list"].apply(lambda x: ast.literal_eval(x))

In [3]:
eu_false_positives = re.compile("|".join(["[furniture|nail] polish",
                                         "irish [whiskey|cream]",
                                         "speak[er|ing]",
                                         "language",
                                         "hispanic",
                                         "tour de france",
                                         "en\w+es\w+pt", # uses pt and es country codes because all segments ar multilingual
                                         ]))

eu_tags_filtered = eu_tags[eu_tags.swifter.apply(lambda x: not bool(re.search(eu_false_positives, x["name"])), axis = 1)]

Pandas Apply:   0%|          | 0/40398 [00:00<?, ?it/s]

In [4]:
tree = {}

for item, row in eu_tags_filtered.iterrows():
    segment_list = row["name_list"]
    level = tree
    for item in segment_list:
        if not item in level:
            level[item] = {}
        level = level[item]

In [5]:
import json
with open("hierarchy.json", "w") as f:
    json.dump(tree, f)

In [6]:
def squeeze_tree(tree, thresh):
    items = list(tree.items())
    if len(items) == 1:
        k,v = items[0]

        if len(v) == 0:
            return k
        
        ret = squeeze_tree(v, thresh)
        if type(ret) == str:
            return " - ".join([k, ret])
        else:
            return {k: ret}

    temp = {}
    for k, v in items:
        temp[k] = squeeze_tree(v, thresh)
    return temp

In [7]:
tree_squeezed = squeeze_tree(tree, 1)
with open("hierarchy.json", "w") as f:
    json.dump(tree_squeezed, f)